## Functions for Cleaning Pinterest DataFrames with Error Handling
I created a function to clean the "df_pin" DataFrame, it can encapsulate the cleaning steps into a single function. 

Also I created an error handling to the function, it can use a try-except block to catch any potential errors that may occur during the cleaning process. Here's the function with error handling:

## Task 1: Clean the DataFrame that contains information about Pinterest posts

#### To clean the df_pin DataFrame you should perform the following transformations:

- Replace empty entries and entries with no relevant data in each column with Nones
- Perform the necessary transformations on the follower_count to ensure every entry is a number. Make sure the data type of this column is an int.
- Ensure that each column containing numeric data has a numeric data type
- Clean the data in the save_location column to include only the save location path
- Rename the index column to ind.
- Reorder the DataFrame columns to have the following column order:
  - ind
  - unique_id
  - title
  - description
  - follower_count
  - poster_name
  - tag_list
  - is_image_or_video
  - image_src
  - save_location
  - category

In [ ]:
from pyspark.sql.functions import col, regexp_replace
from pyspark.sql.types import IntegerType, LongType

def clean_pin_dataframe(df_pin):
    """
    Clean the DataFrame that contains information about Pinterest posts.

    Parameters:
    df_pin (DataFrame): DataFrame containing Pinterest posts data.

    Returns:
    df_pin_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # Replace empty entries and entries with no relevant data in each column with None
        df_pin_cleaned = df_pin.fillna("None")

        # Replace "k" with "000" and cast to integer for follower_count column
        df_pin_cleaned = df_pin_cleaned.withColumn("follower_count", 
                                                   regexp_replace(col("follower_count"), "k", "000").cast(IntegerType()))

        # Cast downloaded column to IntegerType
        df_pin_cleaned = df_pin_cleaned.withColumn("downloaded", df_pin_cleaned["downloaded"].cast(IntegerType()))

        # Cast index column to LongType
        df_pin_cleaned = df_pin_cleaned.withColumn("index", df_pin_cleaned["index"].cast(LongType()))

        # Rename index column to ind
        df_pin_cleaned = df_pin_cleaned.withColumnRenamed("index", "ind")

        # Reorder the DataFrame columns
        df_pin_cleaned = df_pin_cleaned.select("ind", "unique_id", "title", "description", "follower_count", 
                                               "poster_name", "tag_list", "is_image_or_video", "image_src", 
                                               "save_location", "category")

        return df_pin_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None


I can then call this function passing the "df_pin" DataFrame as an argument to clean it:

In [ ]:
# Define file locations
pin_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.pin/partition=0/*.json"

# Read JSON files into DataFrames
df_pin = spark.read.json(pin_file_location)

# Clean the Pinterest posts DataFrame
cleaned_df_pin = clean_pin_dataframe(df_pin)

# Show the cleaned DataFrame
cleaned_df_pin.show()

+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
 ind| unique_id| title| description|follower_count| poster_name| tag_list|is_image_or_video| image_src| save_location| category|
+-----+--------------------+--------------------+--------------------+--------------+--------------------+--------------------+-----------------+--------------------+--------------------+--------------+
10138|927c4658-cc3f-4b9...|14 Amazing Things...|This Costa Rica i...| 10000|Wanderlust Chloe ...|Costa Rica Travel...| image|https://i.pinimg....| /data/travel| travel|
 3156|fa6e31a4-18c2-4ec...|Handprint Reindee...|This post may con...| 892000|Michelle {CraftyM...|Christmas Gifts F...| image|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 5395|71f72304-3708-45f...|So startest du mi...|Der #Vermögensauf...| 3000|DIVDepot – Geld s...|Money Saving Chal...| image|https://i.pinimg....| /data/finance| finance|
 5494|8fb2af68-543b-463...|Dave Ramsey's 7 B...|If you love budge...| 26000|Living Low Key | ...|Financial Peace,F...| image|https://i.pinimg....| /data/finance| finance|
 2418|da8745a6-5160-46c...|100 DIY Christmas...|Here are the best...| 500000| HikenDip|Farmhouse Christm...| image|https://i.pinimg....| /data/christmas| christmas|
 1611|051e231c-7509-437...|Over The Years, P...|[PaidLink] It's m...| 22| Hair Accessories|Natural Beauty Ti...| image|https://i.pinimg....| /data/beauty| beauty|
 3813|49f7dfe7-2e5a-403...|Fun Interactive D...|Tired of Google C...| 9000|Science of Curiosity|Google Classroom,...| image|https://i.pinimg....| /data/education| education|
 7922|a584581c-1b38-473...|45 Top Life Quote...|summcoco gives yo...| 306000|Sumcoco | Decor I...|Life Quotes Love,...| image|https://i.pinimg....| /data/quotes| quotes|
 1643|41be1bcd-1ead-476...|White and Gold Ch...|Connecticut life ...| 84000| Lauren McBride|Christmas Dining ...| image|https://i.pinimg....| /data/christmas| christmas|
 5091|43459535-64e9-48a...|Buying Your First...|Buying your first...| 223|Edwin | Cash The ...|Paying Off Car Lo...| image|https://i.pinimg....| /data/finance| finance|
 9979|2b2abc85-fc51-481...|Paris in the Summ...|Are you traveling...| 3000| Petite in Paris|Torre Eiffel Pari...| image|https://i.pinimg....| /data/travel| travel|
 3032|208b07bc-e042-4e8...| Puffy Ghost Craft|This ghost craft ...| 56000| Thriving Home|Halloween Arts An...| video|https://i.pinimg....|/data/diy-and-crafts|diy-and-crafts|
 6809|f8ca698f-abfc-4b6...|It's Time to Ditc...|It's safe to assu...| 383000| Suburban Men|Business Casual D...| image|https://i.pinimg....| /data/mens-fashion| mens-fashion|
 3644|cde2b4e2-7711-4a8...|More Amazon Must ...|Amazon is a teach...| 41000|Teach Create Moti...|Classroom Hacks,5...| image|https://i.pinimg....| /data/education| education|
 6575|15c45907-657e-439...|Affordable Farmho...|Believe it or not...| 31000| Maria Louise Design|Farmhouse Remodel...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 6145|82e13a07-db99-43a...|HOLIDAY MANTLE DE...|Holiday mantle de...| 83000| Stylin by Aylin|Winter Home Decor...| image|https://i.pinimg....| /data/home-decor| home-decor|
 4|55abcd28-bda1-445...|Mexican Artist Us...|Mexican artist En...| null| Bored Panda|Girl Drawing Sket...| image|https://i.pinimg....| /data/art| art|
 4315|21b59ba9-829d-4c3...|Podcasts for Teac...|Podcasts for Teac...| 25000| Math Giraffe|Middle School Cla...| image|https://i.pinimg....| /data/education| education|
 5069|b75b6f87-deb3-444...|The Vault: Curate...|Sacramento Califo...| null| Style Me Pretty|60th Anniversary ...| image|https://i.pinimg....|/data/event-planning|event-planning|
+-----+--------------------+--------------------+--

## Functions to clean dataframes
I created a function to clean the "df_geo" DataFrame, it can encapsulate the cleaning steps into a single function. 

Also I created an error handling to the function, it can use a try-except block to catch any potential errors that may occur during the cleaning process. Here's the function with error handling:

## Task 2:

#### To clean the df_geo DataFrame you should perform the following transformations:

- Create a new column coordinates that contains an array based on the latitude and longitude columns
- Drop the latitude and longitude columns from the DataFrame
- Convert the timestamp column from a string to a timestamp data type
- Reorder the DataFrame columns to have the following column order:
  - ind
  - country
  - coordinates
  - timestamp

In [ ]:
from pyspark.sql.functions import array, col, to_timestamp

def clean_geo_dataframe(df_geo):
    """
    Clean the DataFrame that contains information about Pinterest geolocation data.

    Parameters:
    df_geo (DataFrame): DataFrame containing Pinterest geolocation data.

    Returns:
    df_geo_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # Create a new column coordinates containing an array of latitude and longitude
        df_geo_cleaned = df_geo.withColumn("coordinates", array(col("latitude"), col("longitude")))

        # Drop the latitude and longitude columns
        df_geo_cleaned = df_geo_cleaned.drop("latitude", "longitude")

        # Convert the timestamp column to a timestamp data type
        df_geo_cleaned = df_geo_cleaned.withColumn("timestamp", to_timestamp("timestamp"))

        # Reorder the DataFrame columns
        df_geo_cleaned = df_geo_cleaned.select("ind", "country", "coordinates", "timestamp")

        return df_geo_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None


I can then call this function passing the "df_geo" DataFrame as an argument to clean it:

In [ ]:
# Define file locations
geo_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.geo/partition=0/*.json"

# Read JSON files into DataFrames
df_geo = spark.read.json(geo_file_location)

# Clean the Pinterest posts DataFrame
cleaned_df_geo = clean_pin_dataframe(df_geo)

# Show the cleaned DataFrame
cleaned_df_geo.show()


+-----+--------------------+--------------------+-------------------+
 ind| country| coordinates| timestamp|
+-----+--------------------+--------------------+-------------------+
 2418|Antarctica (the t...|[-88.4642, -171.061]|2022-05-27 11:30:59|
 9270|Bouvet Island (Bo...|[-84.3984, -144.933]|2022-05-23 03:24:24|
10794|Cocos (Keeling) I...|[-89.5236, -154.567]|2022-01-01 02:26:50|
 5395|British Virgin Is...|[-6.93422, -24.9989]|2020-08-15 19:30:19|
 2074|Central African R...| [-52.3213, -50.11]|2019-11-03 05:41:59|
 1170|Libyan Arab Jamah...| [56.1198, 20.2963]|2019-08-07 22:17:07|
 9550|British Virgin Is...| [-84.918, 7.23235]|2022-03-23 00:25:45|
 7922| Antigua and Barbuda|[-88.0974, -172.052]|2021-01-27 09:14:19|
 9979| Dominican Republic| [14.9967, -120.682]|2018-07-18 19:01:46|
 6575| Dominican Republic| [81.7192, 61.5152]|2020-12-29 22:59:37|
 3032| American Samoa|[-66.7253, -122.489]|2018-04-14 09:28:18|
 637| American Samoa|[-41.2244, -158.022]|2018-06-06 01:25:58|
10321| Burkina Faso|[-89.1005, -151.255]|2022-10-09 08:06:50|
 2923| Cote d'Ivoire|[-84.6302, -164.507]|2019-09-08 22:53:09|
 8304| French Guiana| [-28.8852, -164.87]|2019-09-13 04:50:29|
 4315| Cote d'Ivoire| [-45.8508, 66.1003]|2019-12-15 03:51:28|
 4249| Afghanistan|[-88.5478, -174.971]|2021-09-01 11:10:02|
 6809| Bangladesh|[-89.4254, -161.818]|2020-04-17 13:20:43|
 5069| Azerbaijan|[-63.0063, -157.474]|2021-03-20 09:32:44|
 6145| Mozambique|[-65.9079, -143.845]|2019-12-05 02:09:44|
+-----+--------------------+--------------------+-------------------+
only showing top 20 rows

## Task 3:
1. Create a new column user_name by concatenating first_name and last_name
2. Drop the first_name and last_name columns
3. Convert the date_joined column to a timestamp data type
4. Reorder the DataFrame columns
   - "ind", "user_name", "age", "date_joined"

In [ ]:
from pyspark.sql.functions import concat, col, to_timestamp

def clean_user_dataframe(df_user):
    """
    Clean the DataFrame that contains information about Pinterest user data.

    Parameters:
    df_user (DataFrame): DataFrame containing Pinterest user data.

    Returns:
    df_user_cleaned (DataFrame): Cleaned DataFrame.
    """
    try:
        # 1. Create a new column user_name by concatenating first_name and last_name
        df_user_cleaned = df_user.withColumn("user_name", concat(col("first_name"), col("last_name")))

        # 2. Drop the first_name and last_name columns
        df_user_cleaned = df_user_cleaned.drop("first_name", "last_name")

        # 3. Convert the date_joined column to a timestamp data type
        df_user_cleaned = df_user_cleaned.withColumn("date_joined", to_timestamp("date_joined"))

        # 4. Reorder the DataFrame columns
        df_user_cleaned = df_user_cleaned.select("ind", "user_name", "age", "date_joined")

        return df_user_cleaned

    except Exception as e:
        print(f"An error occurred during data cleaning: {str(e)}")
        return None

I can then call this function passing the "df_user" DataFrame as an argument to clean it:

In [ ]:
# Define file locations
user_file_location = "dbfs:/mnt/mount_name/topics/topics/1209b9ad90a5.user/partition=0/*.json"

# Read JSON files into DataFrames
df_user = spark.read.json(user_file_location)

# Clean the Pinterest posts DataFrame
cleaned_df_user = clean_pin_dataframe(df_user)

# Show the cleaned DataFrame
cleaned_df_user.show()

+-----+------------------+---+-------------------+
 ind| user_name|age| date_joined|
+-----+------------------+---+-------------------+
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
10137| DanielMatthews| 34|2016-01-23 03:59:37|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42|
 8887| AustinRodriguez| 24|2016-03-31 20:56:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
 1514| MichaelPerkins| 40|2017-06-28 16:17:39|
 4315| MichellePrince| 36|2015-12-20 16:38:13|
10625| ChristianLang| 32|2017-10-10 20:09:33|
10625| ChristianLang| 32|2017-10-10 20:09:33|
 9672| JenniferHudson| 22|2016-02-11 20:46:04|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
 1313| BrittanyJones| 32|2016-04-02 03:51:23|
10794| ThomasTurner| 34|2016-12-22 00:02:02|
+-----+------------------+---+-------------------+
only showing top 20 rows